In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [9]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [10]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [11]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 176 ( 0.5%) Users that have less than 1 test interactions


In [13]:
from libs.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

In [14]:
results = pd.DataFrame()

In [15]:
MIN_RATING = 0.0

In [17]:
save_path = "result_experiments/RP3betaRecommender/"

In [19]:
if os.path.exists(save_path+"results.csv"):
    results = pd.read_csv(save_path+"results.csv")
else:
    os.makedirs(save_path)

In [34]:
results[results['MAP'] >= 0.055]

,alpha,beta,min_rating,topK,implicit,normalize_similarity,time,cutoff,PRECISION,PRECISION_RECALL_MIN_DEN,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
3255,0.4,0.25,0.0,10,False,True,2024-11-26 20:19:06,10,0.102753,0.154116,...,0.995075,0.542478,0.995075,0.287698,13.727939,0.999911,0.473037,0.929977,1.116006,0.249313


In [35]:
model_rp3 = RP3betaRecommender(URM_all)
model_rp3.fit(alpha=0.4, beta=0.25, topK=10, min_rating=0.0,
                                      implicit=False, normalize_similarity=True)

RP3betaRecommender: Similarity column 38121 (100.0%), 3752.68 column/sec. Elapsed time 10.16 sec


In [37]:
metrics_df, _ = evaluator_validation.evaluateRecommender(model_rp3)


EvaluatorHoldout: Processed 35560 (100.0%) in 8.90 sec. Users per second: 3996


In [38]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [39]:
write_submission_lib_model(model_rp3, filename='rp3_10.csv')

In [ ]:
max_MAP = -1

In [32]:
model_rp3 = RP3betaRecommender(URM_train)


for alpha in [0.4, 0.5, 0.6, 0.7]:
    for beta in [0.25, 0.3, 0.35]:
        for min_rating in [0.0]:
            for topK in [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50]:
                for implicit in [False]:
                    for normalize_similarity in [True]:
                        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        params = {
                            "alpha": alpha,
                            "beta": beta,
                            "min_rating": min_rating,
                            "topK": topK,
                            "implicit": implicit,
                            "normalize_similarity": normalize_similarity,
                            "time": current_timestamp
                        }
                        
                        model_rp3.fit(alpha=alpha, beta=beta, topK=topK, min_rating=min_rating,
                                      implicit=implicit, normalize_similarity=normalize_similarity)
                        metrics_df, _ = evaluator_validation.evaluateRecommender(model_rp3)
                        if max_MAP < metrics_df["MAP"][10]:
                            max_MAP = metrics_df["MAP"][10]
                        print(', '.join(f"{key}: {value}" for key, value in params.items()), "MAP: ", metrics_df["MAP"][10], "max MAP: ", max_MAP)
                              
                        params_df = pd.DataFrame([params])
                        iteration_results = pd.concat([params_df, metrics_df.reset_index()], axis=1)
                        results = pd.concat([results, iteration_results], ignore_index=True)
                        results.to_csv(save_path+"results.csv", index=False)
                        

RP3betaRecommender: Similarity column 38121 (100.0%), 2166.73 column/sec. Elapsed time 17.59 sec
EvaluatorHoldout: Processed 35560 (100.0%) in 9.00 sec. Users per second: 3953
alpha: 0.4, beta: 0.25, min_rating: 0.0, topK: 2, implicit: False, normalize_similarity: True, time: 2024-11-26 20:16:15 MAP:  0.03722733653828885 max MAP:  0.05487834779580935
RP3betaRecommender: Similarity column 38121 (100.0%), 1559.55 column/sec. Elapsed time 24.44 sec
EvaluatorHoldout: Processed 35560 (100.0%) in 8.71 sec. Users per second: 4083
alpha: 0.4, beta: 0.25, min_rating: 0.0, topK: 3, implicit: False, normalize_similarity: True, time: 2024-11-26 20:16:42 MAP:  0.044929763243878575 max MAP:  0.05487834779580935
RP3betaRecommender: Similarity column 38121 (100.0%), 4251.85 column/sec. Elapsed time 8.97 sec
EvaluatorHoldout: Processed 35560 (100.0%) in 8.72 sec. Users per second: 4076
alpha: 0.4, beta: 0.25, min_rating: 0.0, topK: 4, implicit: False, normalize_similarity: True, time: 2024-11-26 20:17: